<a href="https://colab.research.google.com/github/KimMinoo/Capstone-Design/blob/main/yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Yolov8 학습 코드

In [ ]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics
%pip install -e .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO
from multiprocessing import freeze_support

In [ ]:
if __name__ == '__main__':
    freeze_support()
    try:
        # Load a model
        model = YOLO('yolov8n.pt')  # build a new model from scratch
        model.train(data="/content/drive/MyDrive/yolov8_data/mvp2.yaml", pretrained=True, patience=800 ,epochs=800, optimizer = 'Adam', imgsz=720, batch=8, lr0 = 0.001, lrf=0.001, project="/content/drive/MyDrive/yolov8")  # train the model
    except Exception as e:
        #failList.append(1)
        print("An exception was raised:", e)

Yolov8 Detection 코드

In [ ]:
import torch
from IPython.display import Image, clear_output
import cv2
import os

In [ ]:
model = YOLO('/content/drive/MyDrive/yolov8/train2/weights/best.pt')

directory = '/content/drive/MyDrive/yolov3/check_image'#evaluation할 이미지 위치

model.predict(directory, save=True, save_txt=True, imgsz=720, conf=0.5)

import glob
from google.colab.patches import cv2_imshow
img_list = glob.glob("/content/drive/MyDrive/yolov8/detect/predict/*.jpg")
for i in range(len(img_list)):
  img = cv2.imread(str(img_list[i]))
  cv2_imshow(img)

Yolov8 text 추출 코드

In [ ]:
!pip install easyocr


In [ ]:
import os
import yaml 
directory_path1 = '/usr/local/lib/python3.10/dist-packages/easyocr'
directory_name1 = 'user_network'
full_path1 = os.path.join(directory_path1, directory_name1)
os.makedirs(full_path1, exist_ok=True)

!cp /content/drive/MyDrive/korean_learning/custom.pth /usr/local/lib/python3.10/dist-packages/easyocr/user_network/custom.pth


!cp /content/drive/MyDrive/korean_learning/custom.yaml /usr/local/lib/python3.10/dist-packages/easyocr/user_network/custom.yaml

!cp /content/drive/MyDrive/korean_learning/custom.py /usr/local/lib/python3.10/dist-packages/easyocr/user_network/custom.py

In [ ]:
import easyocr

# reader = easyocr.Reader(['en', 'ko'], gpu=True)

reader = easyocr.Reader(['ko'], gpu=True,
                    model_storage_directory='/usr/local/lib/python3.10/dist-packages/easyocr/user_network',
                    user_network_directory='/usr/local/lib/python3.10/dist-packages/easyocr/user_network',
                    recog_network='custom')


In [ ]:
txtdir = '/content/drive/MyDrive/yolov8/detects/predict/labels/1110077.txt'
imgdir = '/content/drive/MyDrive/yolov8/detects/predict/1110077.jpg'
#imgdir = './runs/detect/predict/1110012.jpg'

def to_int_and_float(label_data):
    label_data[0] = int(label_data[0])    
    label_data[1] = float(label_data[1])
    label_data[2] = float(label_data[2])
    label_data[3] = float(label_data[3])
    label_data[4] = float(label_data[4])
    return label_data


with open(txtdir, 'r') as file:
    vvoundary_datas=[]
    label_datas = []
    # Loop through the lines in the file
    for line in file:
        label_data = list(line.split())
        label_data = to_int_and_float(label_data)
        if label_data[0] == 4:
          vvoundary_datas.append(label_data)
        else:
          label_datas.append(label_data)

vvoundary_datas.sort(key = lambda x:x[2])
label_datas.sort(key = lambda x:x[2])

print(vvoundary_datas)

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
images = cv2.imread(imgdir)

# image = cv2.fastNlMeansDenoisingColored(images, None, 10, 10, 5, 10)
# gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
# ret, image = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
# image = cv2.fastNlMeansDenoising(dst,h=10)

cv2_imshow(image)
cv2.imwrite('/content/folder/image.jpg', image)

image_height, image_width, _ = images.shape
label_class = {0:'vocab', 1:'meaning', 2:'example', 3:'syntax', 4:'vvoundary'}


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.DataFrame(data=[[0, 1, 2]])
data.columns = ["vocab", "meaning", "example"]
vocab_list=[]
meaning_list=[]
example_list=[]

for k,vvoundary_data in enumerate(vvoundary_datas):
    boundary_top = int((vvoundary_data[2] - vvoundary_data[4] / 2) * image_height)
    boundary_bottom = int((vvoundary_data[2] + vvoundary_data[4] / 2) * image_height)
    boundary = {}

    for bbox in label_datas:
        left = int((bbox[1] - bbox[3] / 2) * image_width)
        right = int((bbox[1] + bbox[3] / 2) * image_width)
        top = int((bbox[2] - bbox[4] / 2) * image_height)
        bottom = int((bbox[2] + bbox[4] / 2) * image_height)
        roi = image[top:bottom, left:right]

        if top < boundary_top or bottom > boundary_bottom:
            continue
        
        
        boundary[bbox[0]] = reader.readtext(roi, detail=0)



    class_list = list(boundary.keys())
    class_list.sort()
    if class_list[0] == 0: 
      for i in range(len(class_list)):
        if class_list[i] == 0:
          vocab_list.append(boundary[0])
        elif class_list[i] == 1:
          meaning_list.append(boundary[1])
        elif class_list[i] == 2: 
          example_list.append(boundary[2])
    else:
        if class_list[i] == 0:
          vocab_list.append(vocab_list[-1])
        elif class_list[i] == 1:
          meaning_list.append(meaning_list[-1])
        elif class_list[i] == 2: 
          example_list.append(boundary[2])


for j in range(len(vocab_list)):
  data.loc[j]=[vocab_list[j], meaning_list[j], example_list[j]]

data


In [ ]:
 # data.to_excel('foo.xlsx', sheet_name='Sheet1')